In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import plotly.express as px
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import optuna
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.model_selection import cross_val_score
import plotly.graph_objects as go
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import warnings
from sklearn.model_selection import KFold
warnings.simplefilter(action='ignore', category=FutureWarning)
print("Package Import Successfully")


Package Import Successfully


/Users/jeonjaeyeon/anaconda3/envs/CS2109S/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
original = pd.read_csv('credit_risk_dataset.csv')

In [27]:
print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")

Train shape: (58645, 13)
Test shape: (39098, 12)


In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58645 entries, 0 to 58644
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          58645 non-null  int64  
 1   person_age                  58645 non-null  int64  
 2   person_income               58645 non-null  int64  
 3   person_home_ownership       58645 non-null  object 
 4   person_emp_length           58645 non-null  float64
 5   loan_intent                 58645 non-null  object 
 6   loan_grade                  58645 non-null  object 
 7   loan_amnt                   58645 non-null  int64  
 8   loan_int_rate               58645 non-null  float64
 9   loan_percent_income         58645 non-null  float64
 10  cb_person_default_on_file   58645 non-null  object 
 11  cb_person_cred_hist_length  58645 non-null  int64  
 12  loan_status                 58645 non-null  int64  
dtypes: float64(3), int64(6), object

In [29]:
train = train.loc[:, train.columns != 'id']

In [30]:
train = train.append(original, ignore_index=True)
print("New Train shape:", train.shape)

New Train shape: (91226, 12)


In [31]:
train.isnull().sum()

person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length              895
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 3116
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
loan_status                      0
dtype: int64

In [32]:
train = train.dropna()
train= train.drop_duplicates()
print("Shape of Train data:",train.shape)

Shape of Train data: (87146, 12)


In [33]:
object_columns = ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']

In [16]:
le = LabelEncoder()

for col in object_columns:
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

In [34]:
train.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


In [35]:
train['loan_status'].value_counts()

0    72608
1    14538
Name: loan_status, dtype: int64

In [19]:
X = train.drop(columns=['loan_status'])  
y = train['loan_status'] 

In [20]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [21]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Naive Bayes': GaussianNB(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    'CatBoost': CatBoostClassifier(verbose=0),
    'LightGBM': LGBMClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Gradient Boosting': GradientBoostingClassifier()
}

In [22]:
results = []
accuracies = []
roc_aucs = []

In [36]:
results = [
    {
        'Model': model_name,
        'Accuracy': accuracy_score(y_val, model.fit(X_train, y_train).predict(X_val)),
        'ROC AUC': (
            roc_auc_score(y_val, model.predict_proba(X_val)[:, 1])
            if hasattr(model, "predict_proba")
            else np.nan  # Handle models that don't support probability prediction
        )
    }
    for model_name, model in models.items()
]

# Print results
for result in results:
    print(f"Model: {result['Model']}, Accuracy: {result['Accuracy']:.4f}, ROC AUC: {result['ROC AUC']:.4f}")

/Users/jeonjaeyeon/anaconda3/envs/CS2109S/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning:

[11:22:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.




[LightGBM] [Info] Number of positive: 13147, number of negative: 65284
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000725 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 888
[LightGBM] [Info] Number of data points in the train set: 78431, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.167625 -> initscore=-1.602553
[LightGBM] [Info] Start training from score -1.602553
Model: Logistic Regression, Accuracy: 0.8682, ROC AUC: 0.8490
Model: K-Nearest Neighbors, Accuracy: 0.8854, ROC AUC: 0.8610
Model: Decision Tree, Accuracy: 0.9142, ROC AUC: 0.8482
Model: Random Forest, Accuracy: 0.9478, ROC AUC: 0.9473
Model: Naive Bayes, Accuracy: 0.8700, ROC AUC: 0.8223
Model: XGBoost, Accuracy: 0.9512, ROC AUC: 0.9620
Model: CatBoost, Accuracy: 0.9526, ROC AUC: 0.9605
Model: LightGBM, Accuracy: 0.9503, ROC AUC: 0.9585
M

In [37]:
results_df = pd.DataFrame(results)

In [38]:
fig_acc = px.line(results_df, x='Model', y='Accuracy', title='Model Accuray', markers=True)
fig_acc.update_layout(width=800, height=400)
fig_acc.show()